# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Display the plot
%matplotlib inline

## Imports here
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch import nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data.sampler import SubsetRandomSampler
from collections import OrderedDict

import time

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [ ]:
parent_dir = 'flower_data'
load_only_two_datsets = True

if(True == load_only_two_datsets):
    train_dir = parent_dir + '/train'
    test_dir = parent_dir + '/valid'
    data_dir = {'train' : train_dir, 'test' : test_dir}
else:
    train_dir = parent_dir + '/train'
    valid_dir = parent_dir + '/valid'
    test_dir = parent_dir + '/test'    
    data_dir = {'train' : train_dir, 'valid' : valid_dir, 'test' : test_dir}

In [ ]:
# how many samples per batch to load
batch_size = 32
# percentage of training set to use as validation
valid_size = 0.2

mean_normalize = [0.485, 0.456, 0.406]
std_normalize = [0.229, 0.224, 0.225]

# TODO: Define your transforms for the training, validation and testing sets

data_transforms = {'train': transforms.Compose([transforms.RandomResizedCrop(224),
                                 transforms.RandomRotation(30),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean_normalize, 
                                                      std_normalize)]),
                   'valid': transforms.Compose([transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean_normalize,
                                                      std_normalize)]),
                   'test': transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean_normalize,
                                                     std_normalize)])
                  }



if(True == load_only_two_datsets):
    set_names = ['train', 'train', 'test']
else:    
    set_names = ['train', 'valid', 'test']

# TODO: Load the datasets with ImageFolder
if(True == load_only_two_datsets):
    data = {'train': datasets.ImageFolder(data_dir['train'], transform=data_transforms['train']),
            'test': datasets.ImageFolder(data_dir['test'], transform=data_transforms['test'])}    

    # obtain training indices that will be used for validation
    num_train = len(data['train'])
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(valid_size * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]

    # define samplers for obtaining training and validation batches
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    
    data_loader = {'train': torch.utils.data.DataLoader(data['train'], batch_size=batch_size, sampler=train_sampler),
                   'valid': torch.utils.data.DataLoader(data['train'], batch_size=batch_size, sampler=valid_sampler),
                   'test': torch.utils.data.DataLoader(data['test'], batch_size=batch_size)}
else:
    data = {set_name: datasets.ImageFolder(data_dir[set_name], transform=data_transforms[set_name]) for set_name in set_names}
    # TODO: Using the image datasets and the tranforms, define the dataloaders
    data_loader = {set_name: torch.utils.data.DataLoader(data[set_name], batch_size=batch_size, shuffle=True) for set_name in set_names}

# Get the different classes in the training dataset(should be same for validation and testing dataset)
image_classes = data['train'].classes
# Get number of classes
num_image_classes = len(image_classes)
#print(image_classes)
#print(data['train'].weights)

In [ ]:
def get_counts_per_class(images, num_classes):
    counts_per_class = [0] * num_classes
    #print(count)
    for image in images:
        #print(image[1])
        counts_per_class[image[1]] += 1
    return counts_per_class

# Below function is not correct. Use it only if the function is corrected
def get_weight_per_class(images, num_classes):
    counts_per_class = get_counts_per_class(images, num_classes)
    total_count = float(sum(counts_per_class))
    print(total_count)
    weight_per_class = [0.] * num_classes 
    for idx in range(num_classes):
        weight_per_class[idx] = float(counts_per_class[idx])/total_count
    return weight_per_class

def plot_num_of_data_in_each_class(counts_per_class, classes, plt_title):
    temp_dict = {str(class_number+1): counts_per_class[classes.index(str(class_number+1))] for class_number in range(len(classes))}
    #print(temp_dict, len(temp_dict))
    bar_plot = plt.figure()
    plt.figure(figsize=(20,6))
    plt.bar(range(len(temp_dict)), list(temp_dict.values()), tick_label = list(temp_dict.keys()))
    plt.title(plt_title)
    plt.show()
    
# Get counts of each class in training data        
counts_per_class_train = get_counts_per_class(data['train'].imgs, num_image_classes)
if(False == load_only_two_datsets):
    counts_per_class_valid = get_counts_per_class(data['valid'].imgs, num_image_classes)
counts_per_class_test = get_counts_per_class(data['test'].imgs, num_image_classes)

plot_num_of_data_in_each_class(counts_per_class_train, image_classes, 'Training Dataset')
if(False == load_only_two_datsets):
    plot_num_of_data_in_each_class(counts_per_class_valid, image_classes, 'Validation Dataset')
plot_num_of_data_in_each_class(counts_per_class_test, image_classes, 'Testing Dataset')


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
#print(cat_to_name)
#print(cat_to_name.keys(), len(cat_to_name))

### Visualize Data

In [ ]:
np_mean_normalize = np.asarray(mean_normalize)
np_std_normalize = np.asarray(std_normalize)

In [ ]:
# helper function to un-normalize and display an image
def imshow(img):
    img = img.numpy() # convert image to numpy for display
    img = img.transpose((1, 2, 0))
    img = (img * np_std_normalize) + np_mean_normalize  # unnormalize    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    img = np.clip(img, 0, 1)
    plt.imshow(img)

In [ ]:
# obtain one batch of training images
dataiter = iter(data_loader['test'])
images, labels = dataiter.next()
#print(labels)
#print([image_classes[label] for label in labels])

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
# display 20 images
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(labels[idx].numpy())
    ax.set_title(cat_to_name[str(image_classes[labels[idx]])])

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [ ]:
# get a pre-trained network
model = models.vgg19_bn(pretrained=True)
model

In [ ]:
# TODO: Build and train your network
# Use GPU if it's available
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = torch.device("cpu")

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
# The classifier layer of the pre-trained model has to be updated to match the current
# task of classifying flowers
model.classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 4096)),
                                              ('relu1', nn.ReLU()),
                                              ('drop1', nn.Dropout(0.5)),
                                              ('fc2', nn.Linear(4096, 2048)),
                                              ('relu2', nn.ReLU()),
                                              ('drop2', nn.Dropout(0.5)),
                                              ('fc3', nn.Linear(2048, 102)),
                                              ('output', nn.LogSoftmax(dim=1))]))

print(model.classifier)

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device);

In [ ]:
epochs = 25
print_every_batch = False

start_training = time.time()

epoch_loss_min = np.Inf # track change in loss to save the model accordingly

for epoch in range(epochs):  
    for mode in ['train', 'valid']:
        start_mode = time.time()
        if mode == 'train':
            model.train()
        else:
            model.eval()
            
        running_loss = 0
        running_positives = 0
        
        batch = 0
            
        for inputs, labels in data_loader[mode]:
            if(True == print_every_batch):
                start_batch = time.time()
                
            batch += 1
            #if(0 == batch%4):
                #print(".", end=" ")
                
            # Move input and label tensors to the default device
            inputs, labels = inputs.to(device), labels.to(device)

            # Clear the gradients for every batch
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(mode == 'train'):
                # forward propogation
                logps = model.forward(inputs)
                preds_idx, preds = torch.max(logps, dim=1) 
                loss = criterion(logps, labels)
                
                # Do backward propagation and optimization only in training mode
                if(mode == 'train'):
                    loss.backward()
                    optimizer.step()
                    
            # accumulate loss over every batch
            running_loss += loss.item() #* inputs.size(0)
            #print(preds)
            #print(labels.data)
            #print((preds == labels.data))
            #print(torch.sum(preds == labels.data))
            running_positives += torch.sum(preds == labels.data)
            #print(inputs.size(0))
            #print(running_positives)   
            #print(len(data_loader[mode]))
            
            if(True == print_every_batch):
                print(f"{mode} Batch: {batch}.. " f"Time: {time.time() - start_batch}")
        
        # calculate loss for an epoch
        epoch_loss = running_loss / len(data_loader[mode])
        epoch_acc = running_positives.double() / (len(data_loader[mode])*batch_size)

        print(f"{mode} Epch: {epoch+1}/{epochs}.. "
              f"Loss: {epoch_loss:.3f}.. "
              f"Accu: {epoch_acc:.3f}.. "
              f"Time: {(time.time() - start_mode):.3f}")
        
    #print('\n')
    
    if((mode == 'valid') and (epoch_loss < epoch_loss_min)):
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model ...'.format(
        epoch_loss_min,
        epoch_loss))
        torch.save(model.state_dict(), 'best_model.pt')
        epoch_loss_min = epoch_loss        
        
    print()
        
training_time = time.time() - start_training
print(f"Total training time is {training_time}s ")

# load the best model
model.load_state_dict(torch.load('best_model.pt'))

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
model.class_to_idx = image_datasets['train'].class_to_idx
model.cpu()
torch.save({'model_state_dict': model.state_dict(), 
            'optim_state_dict': optimizer.state_dict(), 
            'class_to_idx': model.class_to_idx}, 'model_checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_model(model_chkpt_path):
    chkpt = torch.load(model_chkpt_path)
    
    model = models.vgg19_bn(pretrained=True)
    
    # Freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False
        
    model.class_to_idx = chkpt['class_to_idx']

    # The classifier layer of the pre-trained model has to be updated to match the current
    # task of classifying flowers
    model.classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 4096)),
                                                  ('relu1', nn.ReLU()),
                                                  ('drop1', nn.Dropout(0.5)),
                                                  ('fc2', nn.Linear(4096, 2048)),
                                                  ('relu2', nn.ReLU()),
                                                  ('drop2', nn.Dropout(0.5)),
                                                  ('fc3', nn.Linear(2048, 102)),
                                                  ('output', nn.LogSoftmax(dim=1))]))
    
    model.load_state_dict(chkpt['model_state_dict'])
    
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
    optimizer.load_state_dict(chkpt['optim_state_dict'])
    
    
    return model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
from PIL import Image

def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    
    # Open the image
    img = Image.open(image)
    
    # Resize
    if img.size[0] > img.size[1]:
        img.thumbnail((10000, 256))
    else:
        img.thumbnail((256, 10000))
        
    # Crop 
    left_margin = (img.width-224)/2
    bottom_margin = (img.height-224)/2
    right_margin = left_margin + 224
    top_margin = bottom_margin + 224
    img = img.crop((left_margin, bottom_margin, right_margin,   
                      top_margin))
    # Normalize
    img = np.array(img)/255
    mean = np.array(mean_normalize) #provided mean
    std = np.array(std_normalize) #provided std
    img = (img - mean)/std
    
    # Move color channels to first dimension as expected by PyTorch
    img = img.transpose((2, 0, 1))
    
    return img

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array(mean_normalize)
    std = np.array(std_normalize)
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes